In [145]:
import warnings
warnings.filterwarnings('ignore')

## Import the dataset

In [146]:
import sqlite3
import pandas as pd

In [147]:
import os

In [148]:
os.listdir()

['tutorial_ipynbs',
 'w2v_practice.model',
 '.gitignore.swp',
 '.gitignore',
 'database.sqlite',
 '.ipynb_checkpoints',
 'KNN_Amazon_fine_food_dataset.ipynb']

In [149]:
db_connection = sqlite3.connect('database.sqlite')

In [150]:
polarisable_dataset = pd.read_sql_query('select * from reviews where Score != 3', db_connection)

In [151]:
polarisable_dataset.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [152]:
polarisable_dataset['Time'].head()

0    1303862400
1    1346976000
2    1219017600
3    1307923200
4    1350777600
Name: Time, dtype: int64

In [153]:
polarisable_dataset.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [154]:
df = polarisable_dataset # just to make typing easier

In [155]:
sampled_df = df.sample(n = 5000, axis = 0, random_state = 0) # random sampling of dataset

In [156]:
sampled_df.shape

(5000, 10)

## Replacing the ratings with 0 (for negative reviews) and 1 (for positive reviews).
#### Score of >3 has been considered as positive and a score of <3 has been taken as negative

In [157]:
type(sampled_df['Score'])

pandas.core.series.Series

In [158]:
scores = sampled_df['Score']

In [159]:
scores[6:12]

161937    5
451123    5
401436    5
396416    5
449554    1
379099    1
Name: Score, dtype: int64

In [160]:
scores = list(map(lambda x: 0 if x<3 else 1, scores))

In [161]:
scores[6:12]

[1, 1, 1, 1, 0, 0]

In [162]:
sampled_df['Score'] = scores

In [163]:
type(sampled_df['Score'].head(2))

pandas.core.series.Series

### Data preprocessing

##### 1. Deduplication
If a user id has multiple entries for the same timestamp, then it should be removed because it is likely that multiple entries at the same timestamp were for the same product of different variety which has a different product id than other variants


In [164]:
sampled_df.duplicated(subset = ['UserId', 'Time']).sum()

62

In [165]:
sampled_deduplicated_df = sampled_df.drop_duplicates(subset = ['UserId', 'Time'], inplace = False, keep = 'first')

#### 2. Extracting the data needed (corpus)
#### And removing html and punctuations

In [166]:
corpus = sampled_deduplicated_df['Text']

In [167]:
# dataset cleaners

import re

def remove_html(sentence):
    html_tag_re_obj = re.compile('<.*>?')
    return re.sub(html_tag_re_obj, ' ', sentence)

def remove_punctuations(sentence):
    cleaned_sentence = re.sub(r'[^a-zA-Z]', r' ', sentence)
    return cleaned_sentence

In [168]:
cleaned_corpus = []
for doc in corpus:
    cleaned_doc_1 = remove_html(doc)
    cleaned_doc_2 = remove_punctuations(doc)
    cleaned_corpus.append(cleaned_doc_2)

#### 3. Removing stop words

In [169]:
## Since the negative food reviews are likely to contain words like "don't", "didn't", etc that impart important
## meaning to the review, we check if such words exist in the corpus that we have. If these words are in the corpus,
## then they should not be in the list of stop words that we use for removing the stopwords from our corpus

count = 0
for doc in cleaned_corpus:
    if "not" in doc:
        count += 1

print(count)

count = 0
for doc in cleaned_corpus:
    if "don't" in doc:
        count += 1

print(count)

count = 0
for doc in cleaned_corpus:
    if "didn't" in doc:
        count += 1

print(count)

1890
0
0


In [170]:
from nltk.corpus import stopwords

In [171]:
stopwords = stopwords.words('english')

In [172]:
stopwords = set(stopwords)

In [173]:
stopwords.remove('not')

In [174]:
'not' in stopwords

False

In [175]:
a = [1,2,3,0,1,0,5]

In [176]:
# filtered_corpus = corpus with docs having no stop words
# doing with the sexy lambda expression

filtered_corpus = list(map(lambda doc: ' '.join(list(filter(lambda word: True if word not in stopwords else False\
                                                            , doc.split()))), corpus))

In [177]:
len(filtered_corpus)

4938

In [178]:
filtered_corpus[:2]

["We pretty much given GF pasta restaurant. It's really good disintegrate toss sauce. My celiac husband thrilled! FYI, first ingredient CORN, quinoa second, not QUITE nutritious might think -- cares!",
 'I bought first SproutMaster years ago Dowling Orchard Market passing Banning, California (back sold them).<br /><br />Through bit pricey plastic tray, I cannot complain ease use quality sprouts grown. The divider IS helpful grow different sprouts, BUT helpful grow type sprout two different stages growth. This way, always fresh sprouts handy anytime.<br /><br />They grow fast Sprout Master makes simple care them.']

In [179]:
### classical way of removing the lambda expressions
### verified the output of lambda expression output with the output of following implementation, outputs are same
# docs_without_stop_words = []
# for i, doc in enumerate(corpus):
#     non_stop_words_in_doc = []
#     for word in doc.split():
#         if word not in stopwords:
#             non_stop_words_in_doc.append(word)
            
    
#     docs_without_stop_words.append(' '.join(non_stop_words_in_doc))

#### 4. Stemming the words (SnowballStemmer)

In [180]:
from nltk.stem import SnowballStemmer

In [181]:
stemmer = SnowballStemmer('english')

In [182]:
stemmed_filtered_corpus = list(map(lambda doc: ' '.join(list(map(stemmer.stem, doc.split()))), corpus))

In [183]:
stemmed_filtered_corpus[:3]

["we had pretti much given up on gf pasta until we had this in a restaurant. it realli good and doesn't disintegr when you toss it with sauce. my celiac husband is thrilled! fyi, the first ingredi is corn, and quinoa is second, so it not quit as nutriti as you might think -- but who cares!",
 'i bought my first sproutmast year ago at the dowl orchard market when pass through banning, california (back when they sold them).<br /><br />through a bit pricey for a plastic tray, i cannot complain about the eas of use or the qualiti of the sprout grown. the divid is help to grow differ sprouts, but it is more help to grow the same type of sprout in two differ stage of growth. this way, you can alway have fresh sprout handi at anytime.<br /><br />they do grow fast and the sprout master make it so simpl to care for them.',
 'for bold coffe fan like us, this is absolut delici coffee. it has a complex taste, a hint of someth resembl red wine. i almost hesit to review it becaus it will probabl cau

## Sorting the dataset according to Time

In [184]:
sampled_deduplicated_df['Text'] = stemmed_filtered_corpus

In [185]:
working_df = sampled_deduplicated_df

In [186]:
working_df_sorted = working_df.sort_values(by = 'Time')

In [187]:
stemmed_filtered_corpus_sorted = working_df_sorted['Text']

## Vectorizing the reviews and splitting into train, cv and test sets and TRAINING and TESTING

### 1.1. Bag of Words (CountVectorizer)

In [188]:
from sklearn.feature_extraction.text import CountVectorizer

In [189]:
count_vectorizer = CountVectorizer()

In [190]:
document_term_matrix = count_vectorizer.fit_transform(stemmed_filtered_corpus_sorted)

In [191]:
document_term_matrix.shape

(4938, 13452)

In [192]:
type(document_term_matrix)

scipy.sparse.csr.csr_matrix

In [193]:
X = document_term_matrix

In [194]:
y = working_df_sorted['Score']

### 1.1.1. Splitting into train, cv and test (Simple Cross Validation)

In [195]:
# from sklearn.model_selection import train_test_split

In [196]:
### This will not work because train_test_split() splits data randomly. What we want is a time-based splitting on
### the dataset that we have sorted chronologically
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) 

In [197]:
print(X.shape, y.shape)

(4938, 13452) (4938,)


In [198]:
type(y)

pandas.core.series.Series

 * ## Function to split dataset into train and test datasets 

In [199]:
def train_test_splitter(X, y, test_size):
    train_size = 1 - test_size
    train_row_upper_index = round(train_size*X.shape[0])
    test_row_lower_index = train_row_upper_index + 1
    
#     print(train_row_upper_index)
    
    X_train = X[:train_row_upper_index + 1, :]
    X_test = X[test_row_lower_index:, :]
    y_train = y.iloc[:train_row_upper_index + 1]
    y_test = y.iloc[test_row_lower_index:]
    
    return X_train, X_test, y_train, y_test

In [200]:
X_train, X_test, y_train, y_test = train_test_splitter(X, y, test_size = 0.25)

In [201]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [202]:
X_train.shape[0] + X_test.shape[0]

4938

In [203]:
### Splitting the previous X_train and y_train into X_train, X_cv and y_train, y_cv
X_train, X_cv, y_train, y_cv = train_test_splitter(X_train, y_train, 0.25)

### 1.1.3. Training and Testing

In [204]:
from sklearn.neighbors import KNeighborsClassifier

In [205]:
from sklearn.metrics import accuracy_score

* ## KNN Classifier and cross validator using Simple Cross Validation

In [206]:
def knn_trainer_and_cross_validator(k, X_train, y_train, X_cv, y_cv):
    
    # Side note: note that X_train and y_train are sparse matrices, and not numpy arrays
    knn = KNeighborsClassifier(k)
    knn.fit(X_train, y_train)
    
    y_pred_cv = knn.predict(X_cv)
    
    accuracy = accuracy_score(y_cv, y_pred_cv) * 100
    
    return accuracy

In [207]:
## tried for k = 1 till 30 and found that for odd k values in this range, accuracy always came below 86%.
## accuracy for k = 31 to 41 (inclusive) was same, 86.8876% and from 43 to 59 it was 86.7435%.
## accuracy for k = 61 to 90 is clearly less than the accuracy obtained for k = 31 to 41.
## we shall choose one of the k values from k = 31 to 41

for i in range(61, 90, 2):
    accuracy = knn_trainer_and_cross_validator(i, X_train, y_train, X_cv, y_cv)
    print('Accuracy for k = ' + str(i) + ' is: ' + str(accuracy))

Accuracy for k = 61 is: 83.13513513513513
Accuracy for k = 63 is: 83.13513513513513
Accuracy for k = 65 is: 83.13513513513513
Accuracy for k = 67 is: 83.13513513513513
Accuracy for k = 69 is: 83.13513513513513
Accuracy for k = 71 is: 83.13513513513513
Accuracy for k = 73 is: 83.13513513513513
Accuracy for k = 75 is: 83.13513513513513
Accuracy for k = 77 is: 83.13513513513513
Accuracy for k = 79 is: 83.13513513513513
Accuracy for k = 81 is: 83.13513513513513
Accuracy for k = 83 is: 83.13513513513513
Accuracy for k = 85 is: 83.13513513513513
Accuracy for k = 87 is: 83.13513513513513
Accuracy for k = 89 is: 83.13513513513513


In [208]:
##### Since the cross-validation accuracy obtained for k = 31, 33, 35, 37, 39 and 41 are same and max, we choose
##### k = 33.

In [209]:
knn = KNeighborsClassifier(n_neighbors = 33)

In [210]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=33, p=2,
           weights='uniform')

In [211]:
### finally testing after tuning the hyperparameter 'k' on the cross validation set
y_pred_test = knn.predict(X_test)

In [212]:
accuracy_score(y_test, y_pred_test)*100

83.29278183292782

### 1.2. Tf-Idf Vectorization

In [213]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [214]:
tfidf_vectorizer = TfidfVectorizer(ngram_range = (1,1))

In [215]:
tfidf_vectorizer = tfidf_vectorizer.fit(stemmed_filtered_corpus_sorted)

In [216]:
# tfidf_vectorizer.vocabulary_

In [217]:
document_term_matrix = tfidf_vectorizer.transform(stemmed_filtered_corpus_sorted)

In [218]:
type(document_term_matrix)

scipy.sparse.csr.csr_matrix

In [219]:
document_term_matrix.shape

(4938, 13452)

In [220]:
X = document_term_matrix

In [221]:
y = working_df_sorted['Score']

In [222]:
X_train_first, X_test, y_train_first, y_test = train_test_splitter(X, y, test_size = 0.25)

In [223]:
X_train, X_cv, y_train, y_cv = train_test_splitter(X_train_first, y_train_first, test_size = 0.25)

### 1.2.1. Training and Testing

In [224]:
accuracies_for_diff_k = []
for i in range(1, 100, 2):
    accuracy = knn_trainer_and_cross_validator(i, X_train, y_train, X_cv, y_cv)
    accuracies_for_diff_k.append(accuracy)
    print('Accuracy for k = ' + str(i) + ' is ' + str(accuracy))

Accuracy for k = 1 is 77.1891891891892
Accuracy for k = 3 is 82.16216216216216
Accuracy for k = 5 is 82.91891891891892
Accuracy for k = 7 is 83.56756756756756
Accuracy for k = 9 is 83.24324324324324
Accuracy for k = 11 is 83.24324324324324
Accuracy for k = 13 is 83.56756756756756
Accuracy for k = 15 is 83.35135135135135
Accuracy for k = 17 is 83.35135135135135
Accuracy for k = 19 is 83.35135135135135
Accuracy for k = 21 is 83.24324324324324
Accuracy for k = 23 is 83.24324324324324
Accuracy for k = 25 is 83.24324324324324
Accuracy for k = 27 is 83.24324324324324
Accuracy for k = 29 is 83.24324324324324
Accuracy for k = 31 is 83.24324324324324
Accuracy for k = 33 is 83.35135135135135
Accuracy for k = 35 is 83.24324324324324
Accuracy for k = 37 is 83.24324324324324
Accuracy for k = 39 is 83.24324324324324
Accuracy for k = 41 is 83.24324324324324
Accuracy for k = 43 is 83.24324324324324
Accuracy for k = 45 is 83.13513513513513
Accuracy for k = 47 is 83.13513513513513
Accuracy for k = 49 is

In [225]:
print(max(accuracies_for_diff_k), accuracies_for_diff_k.index(max(accuracies_for_diff_k)), sep = '\t')

83.56756756756756	3


In [226]:
### index = 3 has the maximum accuracy. This index corresponds to k = 7 (from the above printed list of accuracies)

knn = KNeighborsClassifier(n_neighbors = 7)

In [227]:
## training the final 3-NN

knn = knn.fit(X_train, y_train)

In [228]:
### testing on test data

y_pred = knn.predict(X_test)

In [229]:
### accuracy score

accuracy_score(y_test, y_pred) * 100

83.69829683698296

### 1.3. Average W2V

In [230]:
import gensim

### 1.3.1. Tokenizing each document in the corpus
#### gensim w2v requires each document to be tokenized into words. The corpus will be a list of lists of words

In [231]:
## stemmed_filtered_corpus_sorted is a pandas Series object. It should be converted into a list first.
## after that each sentence in the resulted list should be tokenized into words stored in a list.
## all these lists should be stored into another list so as to give a list of lists as required by gensim w2v

stemmed_filtered_corpus_sorted_list = list(stemmed_filtered_corpus_sorted)

In [232]:
stemmed_filtered_sorted_list_of_tokenized_sentences = []

for sentence in stemmed_filtered_corpus_sorted:
    tokenized_sentence = sentence.split()
    stemmed_filtered_sorted_list_of_tokenized_sentences.append(tokenized_sentence)

In [233]:
w2v = gensim.models.Word2Vec(stemmed_filtered_sorted_list_of_tokenized_sentences, min_count = 1, size = 50, workers = 4)

In [234]:
type(w2v.wv)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [235]:
w2v.wv['happen']

array([ 2.25985467e-01,  3.09546947e-01, -8.32753256e-02,  1.86660707e-01,
        6.68695346e-02, -5.94310045e-01,  3.31842899e-03, -1.12927198e-01,
       -2.23213345e-01,  3.41424882e-01, -2.23198563e-01, -1.28427282e-01,
        2.41897646e-02,  6.68350041e-01,  6.45693839e-02,  8.38903904e-01,
       -7.55395740e-02, -4.95994478e-01,  1.15389526e-01, -3.06253713e-02,
        2.92884022e-01,  3.56055170e-01, -5.18129582e-06,  1.39138281e-01,
       -4.61711198e-01, -1.64292157e-01,  2.61511803e-01, -1.38559699e-01,
       -1.36006325e-01, -3.04406375e-01, -5.75044155e-01,  9.12262052e-02,
       -8.11310485e-02,  5.93134761e-03, -2.07659647e-01,  3.45531613e-01,
        9.52832699e-02, -1.65769048e-02, -2.63604760e-01,  6.45498559e-02,
        3.70814055e-01,  6.73059048e-03, -3.39449018e-01,  1.35534406e-01,
        9.90252593e-05, -2.04053879e-01,  1.70033257e-02,  4.57305282e-01,
       -2.21328050e-01, -2.39042461e-01], dtype=float32)

In [236]:
type(w2v.wv['happen'])

numpy.ndarray

In [237]:
w2v.wv['happen'].shape

(50,)

In [238]:
### saving the w2v model for later use

import os

if(not os.path.exists('w2v_practice.model')):
    w2v.save('w2v_practice.model')
    
else:
    w2v = gensim.models.Word2Vec.load('w2v_practice.model')

In [239]:
### computing avg w2v representation for the reviews dataset

avg_w2v = []
for tokenized_sentence in stemmed_filtered_corpus_sorted_list_of_tokenized_sentences:
    sum_of_vectors_for_each_word = 0
    for word in tokenized_sentence:
        sum_of_vectors_for_each_word += w2v.wv[word]
    avg_w2v.append(sum_of_vectors_for_each_word / len(tokenized_sentence))

In [240]:
type(avg_w2v[0])

numpy.ndarray

In [241]:
avg_w2v[0].shape

(50,)

In [242]:
avg_w2v[0]

array([ 0.661626  ,  0.92628944, -0.013446  ,  0.62764466,  0.18625233,
       -1.4530652 ,  0.15930279, -0.49002123, -0.5853565 ,  0.8513393 ,
       -0.32445616, -0.17482114, -0.07569546,  1.2116948 ,  0.07239655,
        1.691816  ,  0.03975666, -0.9989167 ,  0.23586877, -0.10961175,
        0.56027454,  0.7376916 , -0.17771463,  0.45288604, -1.0219226 ,
       -0.4630284 ,  0.5353912 , -0.05520993, -0.45619234, -0.51378196,
       -1.5669233 ,  0.05079133, -0.03936822,  0.09876107, -0.44751152,
        0.80088794,  0.17019792,  0.03535123, -0.5714997 ,  0.27900317,
        0.9409237 , -0.13131371, -0.5932932 ,  0.17622764, -0.17409271,
       -0.42300984, -0.04031445,  0.95066714, -0.5954353 , -0.60495836],
      dtype=float32)

In [243]:
X = avg_w2v
## y is the same as before

In [244]:
import numpy as np

In [245]:
### the train_test_splitter() assumes X to be a numpy array
X = np.array(avg_w2v)

In [246]:
X.shape

(4938, 50)

In [247]:
X_train_first, X_test, y_train_first, y_test = train_test_splitter(X, y, test_size = 0.25)

In [248]:
X_train, X_cv, y_train, y_cv = train_test_splitter(X_train_first, y_train_first, test_size = 0.25)

### 1.3.2. Training, cross validating to tune hyperparameter k, and then test the model

In [249]:
accuracies_for_diff_k = []
for i in range(1, 100, 2):
    accuracy = knn_trainer_and_cross_validator(i, X_train, y_train, X_cv, y_cv)
    accuracies_for_diff_k.append(accuracy)
    print('Accuracy for k = ' + str(i) + ' is ' + str(accuracy))

Accuracy for k = 1 is 76.21621621621621
Accuracy for k = 3 is 79.56756756756756
Accuracy for k = 5 is 82.37837837837839
Accuracy for k = 7 is 82.05405405405405
Accuracy for k = 9 is 83.45945945945947
Accuracy for k = 11 is 83.56756756756756
Accuracy for k = 13 is 83.13513513513513
Accuracy for k = 15 is 83.13513513513513
Accuracy for k = 17 is 83.13513513513513
Accuracy for k = 19 is 83.13513513513513
Accuracy for k = 21 is 83.13513513513513
Accuracy for k = 23 is 83.02702702702702
Accuracy for k = 25 is 83.13513513513513
Accuracy for k = 27 is 83.13513513513513
Accuracy for k = 29 is 83.13513513513513
Accuracy for k = 31 is 83.13513513513513
Accuracy for k = 33 is 83.13513513513513
Accuracy for k = 35 is 83.13513513513513
Accuracy for k = 37 is 83.13513513513513
Accuracy for k = 39 is 83.13513513513513
Accuracy for k = 41 is 83.13513513513513
Accuracy for k = 43 is 83.13513513513513
Accuracy for k = 45 is 83.13513513513513
Accuracy for k = 47 is 83.13513513513513
Accuracy for k = 49 i

In [250]:
max_accuracy = max(accuracies_for_diff_k)

In [251]:
print(max_accuracy, accuracies_for_diff_k.index(max_accuracy))

83.56756756756756 5


In [252]:
### max_accuracy is indexed at 5 which corresponds to k = 11; hypertuned on CV set. Training knn for k = 5

knn = KNeighborsClassifier(n_neighbors = 5)

In [253]:
knn = knn.fit(X_train, y_train)

### 1.3.3. Testing on test set

In [254]:
y_pred = knn.predict(X_test)

In [255]:
accuracy_score(y_test, y_pred) * 100

80.61638280616383

### 1.4. TfIdf weighted Word2Vec

In [258]:
type(tfidf_vectorizer.vocabulary_)

dict

In [266]:
tfidf_weighted_w2v = []
for sentence in stemmed_filtered_corpus_sorted_list_of_tokenized_sentences:
    tfidf_weighted_sum_of_vectors_for_each_word = 0
    for word in sentence:
        if word not in tfidf_vectorizer.vocabulary_ or word not in w2v.wv:
            continue
        tfidf_weighted_sum_of_vectors_for_each_word += tfidf_vectorizer.vocabulary_[word] * w2v.wv[word]
    tfidf_weighted_w2v.append(tfidf_weighted_sum_of_vectors_for_each_word)

In [267]:
X = np.array(tfidf_weighted_w2v)

In [268]:
X.shape

(4938, 50)

In [282]:
X_train_first, X_test, y_train_first, y_test = train_test_splitter(X, y, test_size = 0.25)

In [283]:
X_train, X_cv, y_train, y_cv = train_test_splitter(X_train_first, y_train_first, test_size = 0.25)

### 1.4.1. Training, tuning hyperparameter k, and testing

In [284]:
accuracies_for_diff_k = []

for i in range(1, 100, 2):
    accuracy = knn_trainer_and_cross_validator(i, X_train, y_train, X_cv, y_cv)
    accuracies_for_diff_k.append(accuracy)
    print('Accuracy for k = ' + str(i) + ' is ' + str(accuracy))

Accuracy for k = 1 is 73.83783783783784
Accuracy for k = 3 is 77.51351351351352
Accuracy for k = 5 is 80.64864864864865
Accuracy for k = 7 is 82.05405405405405
Accuracy for k = 9 is 82.37837837837839
Accuracy for k = 11 is 82.8108108108108
Accuracy for k = 13 is 82.70270270270271
Accuracy for k = 15 is 83.02702702702702
Accuracy for k = 17 is 83.02702702702702
Accuracy for k = 19 is 83.02702702702702
Accuracy for k = 21 is 83.13513513513513
Accuracy for k = 23 is 83.13513513513513
Accuracy for k = 25 is 83.13513513513513
Accuracy for k = 27 is 83.13513513513513
Accuracy for k = 29 is 83.13513513513513
Accuracy for k = 31 is 83.13513513513513
Accuracy for k = 33 is 83.13513513513513
Accuracy for k = 35 is 83.13513513513513
Accuracy for k = 37 is 83.13513513513513
Accuracy for k = 39 is 83.13513513513513
Accuracy for k = 41 is 83.13513513513513
Accuracy for k = 43 is 83.13513513513513
Accuracy for k = 45 is 83.13513513513513
Accuracy for k = 47 is 83.13513513513513
Accuracy for k = 49 is

In [285]:
max_accuracy = max(accuracies_for_diff_k)

In [286]:
print(max_accuracy, accuracies_for_diff_k.index(max_accuracy))

83.13513513513513 10


In [287]:
### index = 10 corresponds to k = 21, ie, hyperparamter k is tuned to 21

knn = KNeighborsClassifier(n_neighbors = 21)

In [288]:
knn = knn.fit(X_train, y_train)

In [289]:
y_pred = knn.predict(X_test)

In [290]:
accuracy_score(y_test, y_pred) * 100

83.29278183292782